In [1]:
import numpy as np
import pandas as pd

## A Multiply indexed Series

### The bad way
(use python dictionary which keys are tuple as index)

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [4]:
pop[('New York',2000):('Texas', 2000)]

(New York, 2000)    18976457
(New York, 2010)    19378102
(Texas, 2000)       20851820
dtype: int64

If we need to select all values from 2010, we have to access the tuple index first, and then access data from 2010.  
This is very inflexible

In [6]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The better way: Pandas MultiIndex

In [7]:
# we can create a multi-index from the tuples as follows

In [8]:
print('index1 :', index)
index = pd.MultiIndex.from_tuples(index)
print('index2 :', index)

index1 : [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
index2 : MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])


In [9]:
# re-index our series
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [11]:
pop[1]

37253956

Some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it 

In [15]:
pop[:,2010]


California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [17]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

### MultiIndex as extra dimension

The `unstack()` method will quickly convert a multiply indexed `Series` into a conventionally indexed `DataFrame`

In [19]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the `stack()` method provides the opposite operation

In [20]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Using multi-index we will have much more flexiblity in the types of data we can represent.  
As multi-index can be used to represent two-dimensional data within a one-dimensional `Series`, we can use it to represent data of three or more dimensions in a `Series` or `DataFrame`

In [22]:
pop_df = pd.DataFrame({'total':pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df     

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

The ufuncs and other functionality also work with hierachical index

In [25]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Creating MultiIndex

The more straightforward way to construct a multiply indexed `Series` or `DataFrame` is to simply pass a list of index arrays

In [27]:
df = pd.DataFrame(
    np.random.rand(4, 2),
    index=[['a', 'a', 'b', 'b'], ['0', '1', '0', '1']],
    columns=['data1', 'data2'])
df

data1     data2
a 0  0.879527  0.084538
  1  0.807221  0.436683
b 0  0.885511  0.592368
  1  0.201530  0.954302

We can also pass a dictionary with appropriate tuples as kes, Pandas will automatically recognize this and use a `MultiIndex` by default

In [29]:
data = {
    ('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102
}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

In [32]:
# from a list of arrays
pd.MultiIndex.from_arrays([['a','a','b','b'], ['0','1','0','1']])


MultiIndex(levels=[['a', 'b'], ['0', '1']],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [34]:
# from a list tuple
pd.MultiIndex.from_tuples([('a',0),('a',1),('b',0),('b',1)])

MultiIndex(levels=[['a', 'b'], [0, 1]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [36]:
# from a Cartesian product of single indices
pd.MultiIndex.from_product([['a','b'],[0,1]])

MultiIndex(levels=[['a', 'b'], [0, 1]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [38]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

### MultiIndex level names

In [40]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

Rows and columns are complete symmetric, so the columns can have multiple levels as well

In [57]:
# Hierachical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
data

array([[55. , 36.4, 26. , 36.7, 36. , 35.8],
       [24. , 37.4, 29. , 35.6, 32. , 36.7],
       [47. , 39.8, 26. , 37.9, 42. , 35.6],
       [45. , 36. , 28. , 37. , 32. , 38. ]])

In [59]:
# create the DataFrame
health_data = pd.DataFrame(data, index=index,
                           columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      55.0  36.4  26.0  36.7  36.0  35.8
     2      24.0  37.4  29.0  35.6  32.0  36.7
2014 1      47.0  39.8  26.0  37.9  42.0  35.6
     2      45.0  36.0  28.0  37.0  32.0  38.0

In [85]:
health_data.loc[::2,'Guido':'Sue']

subject    Guido         Sue      
type          HR  Temp    HR  Temp
year visit                        
2013 1      26.0  36.7  36.0  35.8
2014 1      26.0  37.9  42.0  35.6

## Indexing and Slicing a MultiIndex


### Multiply indexed Series

In [86]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [87]:
# We can access single elements by indexing with multiple terms
pop['California',2010]

37253956

In [88]:
# Partial indexing
# The result is another `Series`, with the lower-level indices maintained
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [94]:
pop.loc['California':'New York', 2000]

state       year
California  2000    33871648
New York    2000    18976457
dtype: int64

In [95]:
pop.iloc[0:2]

state       year
California  2000    33871648
            2010    37253956
dtype: int64

In [97]:
pop[:,2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [100]:
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [101]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

In [103]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      55.0  36.4  26.0  36.7  36.0  35.8
     2      24.0  37.4  29.0  35.6  32.0  36.7
2014 1      47.0  39.8  26.0  37.9  42.0  35.6
     2      45.0  36.0  28.0  37.0  32.0  38.0

Remember that columns are **primary** in a `DataFrame`, and the syntax used for multiply indexed `Series` applies to the columns

In [110]:
health_data['Guido', 'HR']

year  visit
2013  1        26.0
      2        29.0
2014  1        26.0
      2        28.0
Name: (Guido, HR), dtype: float64

In [112]:
# xxx.iloc[rows, columns]
health_data.iloc[:2, :3]

subject      Bob       Guido
type          HR  Temp    HR
year visit                  
2013 1      55.0  36.4  26.0
     2      24.0  37.4  29.0

These indexers provide an array-like view of the underlying two-dimensional data,but each individual index in `loc` or `iloc` can be passed a tuple of multiple indices

In [118]:
health_data.loc[:,('Guido', 'HR')]

year  visit
2013  1        26.0
      2        29.0
2014  1        26.0
      2        28.0
Name: (Guido, HR), dtype: float64

In [121]:
health_data.loc[(2013, 1),('Guido', 'HR')]

26.0

In [123]:
health_data.loc[2013:,(['Guido','Bob'], 'HR')]

subject      Bob Guido
type          HR    HR
year visit            
2013 1      55.0  26.0
     2      24.0  29.0
2014 1      47.0  26.0
     2      45.0  28.0

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error

In [125]:
health_data.loc[(:,1),(:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-125-186aa6537cbc>, line 1)

Getting around this problem by using Pandas's `IndexSlice` object

In [129]:
idx = pd.IndexSlice
health_data.loc[idx[2013,:], idx[:, 'HR']]

subject      Bob Guido   Sue
type          HR    HR    HR
year visit                  
2013 1      55.0  26.0  36.0
     2      24.0  29.0  32.0

## Rearranging Multi-Indices

### Sorted and unsorted indices

Many of the `MultiIndex` slicing operations will `fail` if the index is not sorted

Let's start by creating some simple multiply indexed data where the indices are **not lexographically** sorted

If we try to take a partial slice of this index, it will result in an `error`

In [165]:
# we can see the index has not benn sorted
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names=['char', 'int']
data

char  int
a     1      0.556259
      2      0.223580
c     1      0.208107
      2      0.398452
b     1      0.357763
      2      0.222057
dtype: float64

In [169]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)


<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


For various reasons, partial slicess and other similar operations require the leveks in the `MultiIndex` to be **in sorted order**  
Pandas provides a number of convenience routines to perform this type of sorting.  
e.g.:  
`sort_index()` and `sortlevel()` methods of the `DataFrame`.

In [170]:
data = data.sort_index()
data

char  int
a     1      0.556259
      2      0.223580
b     1      0.357763
      2      0.222057
c     1      0.208107
      2      0.398452
dtype: float64

In [171]:
data['a':'b']

char  int
a     1      0.556259
      2      0.223580
b     1      0.357763
      2      0.222057
dtype: float64

### Stacking and unstacking indices

In [172]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

There are some options specifying the level.


In [177]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [179]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


### Index setting and resetting

In [180]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [182]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [185]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [186]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      55.0  36.4  26.0  36.7  36.0  35.8
     2      24.0  37.4  29.0  35.6  32.0  36.7
2014 1      47.0  39.8  26.0  37.9  42.0  35.6
     2      45.0  36.0  28.0  37.0  32.0  38.0

If we'd like to average-out the measurements in the two visit each year. We can do this by **naming the index level**

In [189]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     39.5  36.9  27.5  36.15  34.0  36.25
2014     46.0  37.9  27.0  37.45  37.0  36.80

In [192]:
health_data.mean(level='visit')

subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
visit                                       
1        51.0  38.1  26.0  37.3  39.0  35.70
2        34.5  36.7  28.5  36.3  32.0  37.35

By further making use of the `aixs` keyword, we can take the mean among levels on the columns as well

In [196]:
health_data.sum(axis=1, level='type')

type           HR   Temp
year visit              
2013 1      117.0  108.9
     2       85.0  109.7
2014 1      115.0  113.3
     2      105.0  111.0

In [197]:
health_data.sum(axis=1, level='subject')

subject      Bob  Guido   Sue
year visit                   
2013 1      91.4   62.7  71.8
     2      61.4   64.6  68.7
2014 1      86.8   63.9  77.6
     2      81.0   65.0  70.0